In [46]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append('..')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [47]:
# from sklearnex import patch_sklearn
# patch_sklearn()

In [48]:
import time

import numpy as np
import pandas as pd

from sklearn.linear_model import RidgeClassifier, LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score, KFold
from src.utils import OffsetScaler, get_fps_offset
import xgboost as xgb
import catboost as cb
import lightgbm as lgb
import seaborn as sns


mae = 'neg_mean_absolute_error'
mse = 'neg_mean_squared_error'
rmse = 'neg_root_mean_squared_error'
roc_auc = 'neg_roc_auc_score'
N_JOBS = 24
RANDOM_SEED = 42

# prepare models
models = {}

models['LR'] = LogisticRegression()
models['Ridge'] = RidgeClassifier()
models['DT'] = DecisionTreeClassifier(random_state=RANDOM_SEED)
# models['Lasso'] = Lass()
models['KNN'] = KNeighborsClassifier(n_jobs=N_JOBS)
models['SVC'] = SVC()
models['RF'] = RandomForestClassifier(random_state=RANDOM_SEED, n_jobs=N_JOBS)
models['XGB'] = xgb.XGBClassifier(random_state=RANDOM_SEED, n_jobs=N_JOBS, verbosity=0)
models['CB'] = cb.CatBoostClassifier(random_seed=RANDOM_SEED, thread_count=N_JOBS, verbose=False)
models['LGB'] = lgb.LGBMClassifier(random_state=RANDOM_SEED, n_jobs=N_JOBS, verbose=0)

In [73]:
def scale(X):
    offset = get_fps_offset(X.columns)
    scaler = OffsetScaler(offset)
    return scaler.fit_transform(X.values)

def eval_model(name, model, X, y):    
    tic = time.time()
        
    kfold = KFold(n_splits=5, shuffle=True, random_state=RANDOM_SEED)
    cv_res = cross_val_score(model, scale(X), y, cv=kfold, scoring='roc_auc')
    
    score = cv_res.mean() - cv_res.std()
    
    toc = time.time()
    print("%7s: %3.3f    (%3.3f ± %3.3f)    %.1fs" % (name, score, cv_res.mean(), cv_res.std(), toc - tic))
    return cv_res 

In [66]:
X = pd.read_pickle('../data/processed/X_train_1.pkl.zip')
y = pd.read_pickle('../data/processed/y_train_1.pkl')

results = {}

for name, model in models.items():
    results[name] = eval_model(name, model, X, y)    
    
results = pd.DataFrame(results)
sns.boxplot(results);

/home/ergot/miniforge3/envs/ml/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/home/ergot/miniforge3/envs/ml/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.

LR: 0.851    (0.858 ± 0.007)    8.1s
Ridge: 0.777    (0.785 ± 0.008)    4.1s
DT: 0.734    (0.750 ± 0.016)    12.5s
KNN: 0.848    (0.855 ± 0.007)    1.0s


KeyboardInterrupt: 

In [ ]:
X = pd.read_pickle('../data/processed/X_train_2.pkl.zip')
y = pd.read_pickle('../data/processed/y_train_2.pkl')

results = {}

for name, model in models.items():
    results[name] = eval_model(name, model, X, y)    
    
results = pd.DataFrame(results)
sns.boxplot(results);

In [ ]:
X = pd.read_pickle('../data/processed/X_train_3.pkl.zip')
y = pd.read_pickle('../data/processed/y_train_3.pkl')

results = {}

for name, model in models.items():
    results[name] = eval_model(name, model, X, y)    
    
results = pd.DataFrame(results)
sns.boxplot(results);

In [68]:
X = pd.read_pickle('../data/processed/X_train_2.pkl.zip')
y = pd.read_pickle('../data/processed/y_train_2.pkl')

In [74]:
cls = xgb.XGBClassifier(random_state=RANDOM_SEED, n_jobs=N_JOBS, verbosity=0,
                        tree_method='gpu_hist',
                        predictor='gpu_predictor',
                        gpu_id=1)

eval_model('xgb_gpu', cls, X, y)

cls = xgb.XGBClassifier(random_state=RANDOM_SEED, n_jobs=N_JOBS, verbosity=0)
eval_model('xgb', cls, X, y);

xgb_gpu: 0.734    (0.766 ± 0.032)    5.5s
    xgb: 0.731    (0.766 ± 0.035)    12.3s


In [ ]:
cls = cb.CatBoostClassifier(random_seed=RANDOM_SEED, thread_count=N_JOBS, verbose=False, task_type="GPU")